In [10]:
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2Model
import torch

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")

In [12]:
print(device)

cpu


In [13]:
model = Blip2Model.from_pretrained("Salesforce/blip2-opt-2.7b")

Loading checkpoint shards: 100%|██████████| 2/2 [02:58<00:00, 89.21s/it] 


In [14]:
model.to(device)

Blip2Model(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((1408,), eps=1e-06, 

In [15]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [16]:
prompt = "Question: how many cats are there? Answer:"
inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

In [18]:
outputs = model(**inputs)

In [33]:
answer_string = processor.decode(outputs.logits.argmax(-1))


TypeError: argument 'ids': 'list' object cannot be interpreted as an integer

In [ ]:
inputs = tokenizer(["a photo of a cat"], padding=True, return_tensors="pt")

In [36]:
from grid.model.perception.captioning.blip2 import BLIPv2
model = BLIPv2()
img_path = os.path.join(data_dir, "rgb.png")
img = np.asarray(Image.open(img_path).convert("RGB"))
outputs = model.run(img, "Describe the image.")

ModuleNotFoundError: No module named 'grid.model'

In [39]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image

# Carregar a imagem localmente
image_path = "./images/blue.jpg"
image = Image.open(image_path).convert("RGB")

# Carregar o processador e o modelo TROCR
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')

# Pré-processar a imagem para extrair os valores de pixel
pixel_values = processor(images=image, return_tensors="pt").pixel_values

# Gerar IDs de tokens para a imagem fornecida
generated_ids = model.generate(pixel_values)

# Decodificar os IDs em texto
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\kennedy.reis\AppData\Local\anaconda3\envs\env_clip\lib\site-packages\transformers\generation\utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [41]:
from transformers import AutoModel, AutoProcessor

model = AutoModel.from_pretrained("unum-cloud/uform-gen2-qwen-500m", trust_remote_code=True)
processor = AutoProcessor.from_pretrained("unum-cloud/uform-gen2-qwen-500m", trust_remote_code=True)

prompt = "Question or Instruction"
image = Image.open("./images/blue.jpg")

inputs = processor(text=[prompt], images=[image], return_tensors="pt")
with torch.inference_mode():
     output = model.generate(
        **inputs,
        do_sample=False,
        use_cache=True,
        max_new_tokens=256,
        eos_token_id=151645,
        pad_token_id=processor.tokenizer.pad_token_id
    )

prompt_len = inputs["input_ids"].shape[1]
decoded_text = processor.batch_decode(output[:, prompt_len:])[0]

c:\Users\kennedy.reis\AppData\Local\anaconda3\envs\env_clip\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kennedy.reis\.cache\huggingface\hub\models--unum-cloud--uform-gen2-qwen-500m. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
A new version of the following files was downloaded from https://h

KeyError: 'qwen2'

In [ ]:
decoded_text